## 9. Testing and Debugging

### 78 Use Mocks to Test Code with Complex Dependencies

In [1]:
import logging

In [2]:
class DatabaseConnection:
    def __init__(self, host, port):
        pass

class DatabaseConnectionError(Exception):
    pass

def get_animals(database, species):
    # Query the Database
    raise DatabaseConnectionError('Not connected')
    # Return a list of (name, last_mealtime) tuples

In [3]:
try:
    database = DatabaseConnection('localhost', '4444')
    
    get_animals(database, 'Meerkat')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-3-ac3dd34f44af>", line 4, in <module>
    get_animals(database, 'Meerkat')
  File "<ipython-input-2-9961780f04ff>", line 10, in get_animals
    raise DatabaseConnectionError('Not connected')
DatabaseConnectionError: Not connected


In [4]:
from datetime import datetime

In [5]:
from unittest.mock import Mock

In [6]:
mock = Mock(spec=get_animals)
expected = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 45)),
]
mock.return_value = expected

In [7]:
try:
    mock.does_not_exist
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-7-d4b191c64618>", line 2, in <module>
    mock.does_not_exist
  File "/usr/local/lib/python3.8/unittest/mock.py", line 637, in __getattr__
    raise AttributeError("Mock object has no attribute %r" % name)
AttributeError: Mock object has no attribute 'does_not_exist'


In [8]:
database = object()
result = mock(database, 'Meerkat')
assert result == expected

In [9]:
mock.assert_called_once_with(database, 'Meerkat')

In [10]:
try:
    mock.assert_called_once_with(database, 'Giraffe')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-10-74361802c8c1>", line 2, in <module>
    mock.assert_called_once_with(database, 'Giraffe')
  File "/usr/local/lib/python3.8/unittest/mock.py", line 925, in assert_called_once_with
    return self.assert_called_with(*args, **kwargs)
  File "/usr/local/lib/python3.8/unittest/mock.py", line 913, in assert_called_with
    raise AssertionError(_error_message()) from cause
AssertionError: expected call not found.
Expected: mock(<object object at 0x7fd4d00b1090>, 'Giraffe')
Actual: mock(<object object at 0x7fd4d00b1090>, 'Meerkat')


In [11]:
from unittest.mock import ANY

In [12]:
mock = Mock(spec=get_animals)
mock('database 1', 'Rabbit')
mock('database 2', 'Bison')
mock('database 3', 'Meerkat')

mock.assert_called_with(ANY, 'Meerkat')

In [13]:
class MyError(Exception):
    pass

try:
    mock = Mock(spec=get_animals)
    mock.side_effect = MyError('Whoops! Big problem')
    result = mock(database, 'Meerkat')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-13-cb28d0154568>", line 7, in <module>
    result = mock(database, 'Meerkat')
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1081, in __call__
    return self._mock_call(*args, **kwargs)
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1085, in _mock_call
    return self._execute_mock_call(*args, **kwargs)
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1140, in _execute_mock_call
    raise effect
MyError: Whoops! Big problem


In [14]:
def get_food_period(database, species):
    # Query the Database
    pass
    # Return a time delta

def feed_animal(database, name, when):
    # Write to the Database
    pass

def do_rounds(database, species):
    now = datetime.datetime.utcnow()
    feeding_timedelta = get_food_period(database, species)
    animals = get_animals(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_animal(database, name, now)
            fed += 1

    return fed

In [15]:
def do_rounds(database, species, *,
              now_func=datetime.utcnow,
              food_func=get_food_period,
              animals_func=get_animals,
              feed_func=feed_animal):
    now = now_func()
    feeding_timedelta = food_func(database, species)
    animals = animals_func(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_func(database, name, now)
            fed += 1

    return fed

In [16]:
from datetime import timedelta

In [17]:
now_func = Mock(spec=datetime.utcnow)
now_func.return_value = datetime(2019, 6, 5, 15, 45)

food_func = Mock(spec=get_food_period)
food_func.return_value = timedelta(hours=3)

animals_func = Mock(spec=get_animals)
animals_func.return_value = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 45)),
]

feed_func = Mock(spec=feed_animal)

In [18]:
result = do_rounds(
    database,
    'Meerkat',
    now_func=now_func,
    food_func=food_func,
    animals_func=animals_func,
    feed_func=feed_func)

assert result == 2

In [19]:
from unittest.mock import call

In [20]:
food_func.assert_called_once_with(database, 'Meerkat')

animals_func.assert_called_once_with(database, 'Meerkat')

feed_func.assert_has_calls(
    [
        call(database, 'Spot', now_func.return_value),
        call(database, 'Fluffy', now_func.return_value),
    ],
    any_order=True)

In [21]:
from unittest.mock import patch

In [22]:
print('Outside patch:', get_animals)

with patch('__main__.get_animals'):
    print('Inside patch: ', get_animals)

print('Outside again:', get_animals)

Outside patch: <function get_animals at 0x7fd4d00e51f0>
Inside patch:  <MagicMock name='get_animals' id='140551780611648'>
Outside again: <function get_animals at 0x7fd4d00e51f0>


In [23]:
try:
    fake_now = datetime(2019, 6, 5, 15, 45)
    
    with patch('datetime.datetime.utcnow'):
        datetime.utcnow.return_value = fake_now
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1490, in __enter__
    setattr(self.target, self.attribute, new_attr)
TypeError: can't set attributes of built-in/extension type 'datetime.datetime'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-23-4cb98598fb2e>", line 4, in <module>
    with patch('datetime.datetime.utcnow'):
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1503, in __enter__
    if not self.__exit__(*sys.exc_info()):
  File "/usr/local/lib/python3.8/unittest/mock.py", line 1509, in __exit__
    setattr(self.target, self.attribute, self.temp_original)
TypeError: can't set attributes of built-in/extension type 'datetime.datetime'


In [24]:
def get_do_rounds_time():
    return datetime.datetime.utcnow()

def do_rounds(database, species):
    now = get_do_rounds_time()

with patch('__main__.get_do_rounds_time'):
    pass

In [25]:
def do_rounds(database, species, *, utcnow=datetime.utcnow):
    now = utcnow()
    feeding_timedelta = get_food_period(database, species)
    animals = get_animals(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_animal(database, name, now)
            fed += 1

    return fed

In [26]:
from unittest.mock import DEFAULT

In [27]:
with patch.multiple('__main__',
                    autospec=True,
                    get_food_period=DEFAULT,
                    get_animals=DEFAULT,
                    feed_animal=DEFAULT):
    now_func = Mock(spec=datetime.utcnow)
    now_func.return_value = datetime(2019, 6, 5, 15, 45)
    get_food_period.return_value = timedelta(hours=3)
    get_animals.return_value = [
        ('Spot', datetime(2019, 6, 5, 11, 15)),
        ('Fluffy', datetime(2019, 6, 5, 12, 30)),
        ('Jojo', datetime(2019, 6, 5, 12, 45))
    ]

    result = do_rounds(database, 'Meerkat', utcnow=now_func)
    assert result == 2

    get_food_period.assert_called_once_with(database, 'Meerkat')
    get_animals.assert_called_once_with(database, 'Meerkat')
    feed_animal.assert_has_calls(
        [
            call(database, 'Spot', now_func.return_value),
            call(database, 'Fluffy', now_func.return_value),
        ],
        any_order=True)

> - `unittest.mock` 모듈은 `Mock` 클래스를 사용해 인터페이스의 동작을 흉내 낼 수 있게 해준다. 테스트를 하루 때 데스트 대상 코드가 호출해야 하는 의존 관계 함수를 설정하기 힘들 경우에는 목을 사용하면 유용하다.
> - 목을 사용할 때는 테스트 대상 코드의 동작을 검증하는 것과 테스트 대상 코드가 호출하는 의존 관계 함수들이 호출되는 방식을 검증하는 것이 모두 중요하다. `Mock.assert_called_once_with`나 이와 비슷한 메서드들을 사용해 이런 검증을 수행한다.
> - 목을 테스트 대상 코드에 주입할 때는 키워드를 사용해 호출해야 하는 인자를 쓰거나, `unittest.mock.patch` 또는 이와 비슷한 메서드들을 사용한다.